# yolov1_implementation

출처 : https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/object_detection/YOLO

1. architecture
2. IOU
3. loss

In [1]:
!pip install torchinfo

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Architecture

모델 부분:

YOLOv1 클래스는 여섯 개의 컨볼루션 블록과 두 개의 전결합층(fully connected)을 정의합니다.
각 컨볼루션 블록은 nn.Conv2d, nn.LeakyReLU(0.1), nn.MaxPool2d 등을 사용하여 구성되어 있습니다.

In [3]:
# 논문에 제시된 아키텍처 구성 (YOLOv1)
architecture_config = [
    (7, 64, 2, 3),       # (kernel_size, filters, stride, padding)
    "M",                 # maxpool
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],  # 해당 블록을 4번 반복
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],  # 해당 블록을 2번 반복
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1)
]

In [4]:
def create_conv_layers(config, in_channels):
    layers = []
    for module in config:
        if type(module) == tuple:
            # 튜플 형태: (kernel_size, filters, stride, padding)
            kernel_size, filters, stride, padding = module
            layers.append(nn.Conv2d(in_channels, filters, kernel_size, stride, padding))
            layers.append(nn.LeakyReLU(0.1))
            in_channels = filters
        elif module == "M":
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        elif type(module) == list:
            # 리스트 형태: [ conv1 튜플, conv2 튜플, 반복 횟수 ]
            conv1, conv2, num_repeats = module
            for _ in range(num_repeats):
                # 첫 번째 컨볼루션
                k, f, s, p = conv1
                layers.append(nn.Conv2d(in_channels, f, k, s, p))
                layers.append(nn.LeakyReLU(0.1))
                in_channels = f
                # 두 번째 컨볼루션
                k, f, s, p = conv2
                layers.append(nn.Conv2d(in_channels, f, k, s, p))
                layers.append(nn.LeakyReLU(0.1))
                in_channels = f
    return nn.Sequential(*layers)

class YOLOv1(nn.Module):
    def __init__(self, in_channels=3, num_classes=20, split_size=7, num_boxes=2):
        super(YOLOv1, self).__init__()
        self.conv_layers = create_conv_layers(architecture_config, in_channels)
        # 입력 이미지가 448x448인 경우, 마지막 컨볼루션 feature map은 7x7 (논문 기준)
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * 7 * 7, 4096),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),  # 논문에서 사용한 dropout
            nn.Linear(4096, split_size * split_size * (num_classes + num_boxes * 5))
        )
        self.split_size = split_size
        self.num_boxes = num_boxes
        self.num_classes = num_classes

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

## IOU

주어진 두 박스의 중심 좌표와 크기를 바탕으로 좌측상단, 우측하단 좌표를 계산하고, 교집합 영역을 통해 IoU를 계산합니다.


In [5]:
# IoU 계산 함수 (YOLOv1에서 사용하는 bounding box 형식: [x_center, y_center, width, height])
def iou(boxes1, boxes2, eps=1e-6):
    """
    boxes1, boxes2: 텐서, 마지막 차원이 [x_center, y_center, width, height]
    """
    # 좌측 상단, 우측 하단 좌표 계산
    box1_x1 = boxes1[..., 0:1] - boxes1[..., 2:3] / 2
    box1_y1 = boxes1[..., 1:2] - boxes1[..., 3:4] / 2
    box1_x2 = boxes1[..., 0:1] + boxes1[..., 2:3] / 2
    box1_y2 = boxes1[..., 1:2] + boxes1[..., 3:4] / 2

    box2_x1 = boxes2[..., 0:1] - boxes2[..., 2:3] / 2
    box2_y1 = boxes2[..., 1:2] - boxes2[..., 3:4] / 2
    box2_x2 = boxes2[..., 0:1] + boxes2[..., 2:3] / 2
    box2_y2 = boxes2[..., 1:2] + boxes2[..., 3:4] / 2

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    inter = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)
    box1_area = torch.abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = torch.abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))
    iou_val = inter / (box1_area + box2_area - inter + eps)
    return iou_val


### NMS

- confidence가 낮은 박스 제거 후, 가장 높은 confidence 박스를 선택하고 나머지 박스들과 IoU를 계산하여 제거합니다.
- 박스 간의 IoU 계산 시, 텐서 변환 및 .item() 호출로 스칼라 값을 사용합니다.

In [6]:
# NMS (Non-Maximum Suppression) 함수: confidence가 낮은 박스와 겹치는 박스 제거
def nms(bboxes, iou_threshold, conf_threshold):
    """
    bboxes: 각 요소가 [pred_class, confidence, x1, y1, x2, y2] 형태인 리스트
    iou_threshold: IoU 임계치
    conf_threshold: confidence 임계치 이하인 박스 제거
    """
    bboxes = [box for box in bboxes if box[1] > conf_threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)
        bboxes = [
            box for box in bboxes
            if box[0] != chosen_box[0] or iou(torch.tensor(chosen_box[2:]).unsqueeze(0),
                                                 torch.tensor(box[2:]).unsqueeze(0)).item() < iou_threshold
        ]
        bboxes_nms.append(chosen_box)
    return bboxes_nms

## Loss

- 예측값을 [batch, 7, 7, 30] 형태로 재구성한 후, 각 grid cell마다 두 박스의 IoU를 계산하고, best box 선택, 그리고 좌표, confidence, 클래스 손실을 각각 계산하여 최종 loss를 반환합니다.


In [7]:
# Loss 함수 (YOLOv1 손실 함수 구현)
def yolo_loss(predictions, target, S=7, B=2, C=20, lambda_coord=5, lambda_noobj=0.5):
    """
    predictions: [batch, 7*7*30] 형태의 텐서
    target: [batch, 7, 7, 30] 형태의 텐서
    """
    predictions = predictions.view(-1, S, S, C + B * 5)
    # 두 박스에 대해 IoU 계산 (bbox 좌표 부분: 21:25와 26:30)
    iou_b1 = iou(predictions[..., 21:25], target[..., 21:25])
    iou_b2 = iou(predictions[..., 26:30], target[..., 21:25])
    ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

    iou_maxes, bestbox = torch.max(ious, dim=0)
    bestbox = bestbox.float().unsqueeze(-1)

    exists_box = target[..., 20].unsqueeze(-1)

    # Box coordinate loss (좌표 손실)
    box_pred = exists_box * (bestbox * predictions[..., 26:30] + (1 - bestbox) * predictions[..., 21:25])
    box_target = exists_box * target[..., 21:25]

    # width, height에 sqrt 적용 (음수 방지를 위해 클램핑)
    box_pred_wh = torch.sqrt(torch.clamp(box_pred[..., 2:4], min=1e-6))
    box_target_wh = torch.sqrt(torch.clamp(box_target[..., 2:4], min=1e-6))
    box_pred = torch.cat([box_pred[..., :2], box_pred_wh], dim=-1)
    box_target = torch.cat([box_target[..., :2], box_target_wh], dim=-1)

    box_loss = torch.sum((box_pred - box_target) ** 2)

    # Object loss (물체가 있을 때 confidence 손실)
    pred_conf = bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
    confidence_target = exists_box * iou_maxes.unsqueeze(-1)
    object_loss = torch.sum((exists_box * (pred_conf - confidence_target)) ** 2)

    # No-object loss (물체가 없을 때 confidence 손실)
    noobj_loss = torch.sum(((1 - exists_box) * predictions[..., 20:21]) ** 2)
    noobj_loss += torch.sum(((1 - exists_box) * predictions[..., 25:26]) ** 2)

    # Class loss (클래스 확률 손실)
    class_loss = torch.sum((exists_box * (predictions[..., :20] - target[..., :20])) ** 2)

    total_loss = lambda_coord * box_loss + object_loss + lambda_noobj * noobj_loss + class_loss
    return total_loss

In [8]:
from torchinfo import summary

# YOLOv1 모델 인스턴스 생성 (입력 채널: 3, 이미지 크기: 448x448)
model = YOLOv1(in_channels=3, num_classes=20, split_size=7, num_boxes=2)

# 모델 요약 정보 출력 (배치 크기 1, 입력 크기 3x448x448)
summary(model, input_size=(1, 3, 448, 448))


Layer (type:depth-idx)                   Output Shape              Param #
YOLOv1                                   [1, 1470]                 --
├─Sequential: 1-1                        [1, 1024, 7, 7]           --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         9,472
│    └─LeakyReLU: 2-2                    [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-3                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-4                       [1, 192, 112, 112]        110,784
│    └─LeakyReLU: 2-5                    [1, 192, 112, 112]        --
│    └─MaxPool2d: 2-6                    [1, 192, 56, 56]          --
│    └─Conv2d: 2-7                       [1, 128, 56, 56]          24,704
│    └─LeakyReLU: 2-8                    [1, 128, 56, 56]          --
│    └─Conv2d: 2-9                       [1, 256, 56, 56]          295,168
│    └─LeakyReLU: 2-10                   [1, 256, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          6